In [ ]:
import pandas as pd
import json
import requests

# Read the data
df = pd.read_csv("../data_old/adult/generalization/adult_train.csv")
# df = pd.read_csv("../data_old/adult/specalization/native-country_vorverarbeitet.csv")

# remove 'record_id' and 'income' columns if it exists
if 'record_id' in df.columns:
    df = df.drop(columns=['record_id'])
if 'income' in df.columns:
    df = df.drop(columns=['income'])

# selected_columns = ['native-country']  # Specify the columns to analyze
selected_columns = []  # If you want to analyze all columns, leave this empty

if not selected_columns:
  selected_columns = df.columns.tolist()

# Gather distinct values for each selected column
# Multiple columns: list of lists
values = []
for col in selected_columns:
  col_values = sorted(df[col].unique().tolist())
  values.append(col_values)
  print(f"Distinct values in column '{col}':")
  print(col_values)
  print(f"Number of unique values in '{col}': {len(col_values)}")

# Compose a prompt for the LLM
# list of lists with multiple columns
multi_column_prompt = f"""
You are given a list of lists, where each inner list contains the distinct values from a single column of a dataset. The column names are unknown.

Here are the column value lists:

{values}

Your task is to construct a generalization hierarchy for each column independently, using only the values present in each list. You must not use any external knowledge, and should make no assumptions about the meanings of the values.

Instructions:

- For each column:
  - Create a nested Python dictionary representing a generalization hierarchy.
    - Each key is a value from that column.
    - Each value is a dictionary of its children (more specific values).
    - Leaf nodes must have empty dictionaries (`{{}}`).
  - Group values under others only if their naming or structure suggests a hierarchical relationship (e.g., shared tokens, substrings, prefixes, suffixes, or delimiters).
  - Each value must appear exactly once in the hierarchy.
  - Top-level values should be the most general (parents).
  - Do not invent or introduce any new categories that are not part of the observed values.
  - If a value represents a missing or unknown entry (e.g., '?'), treat it as the most general level in the hierarchy — it may contain any other value as a child, but may not be a child itself.

- Return a single Python dictionary where:
  - Keys are column indexes (starting from `0`).
  - Values are the corresponding hierarchy dictionaries.

Return only the resulting dictionary. Do not return any explanation or code.
""".strip()

print("\nPrompt for LLM:")
print(multi_column_prompt)  

# Query the local Ollama LLM (assumes Ollama is running locally with a suitable model, e.g., llama3)
ollama_url = "http://localhost:11434/api/generate"
ollama_payload = {
    "model": "llama3.2",  # or another model you have installed, we however used ChatGPT-4o with the final prompts‚
    "prompt": multi_column_prompt,
    "stream": False
}
response = requests.post(ollama_url, json=ollama_payload)
if response.status_code != 200:
    raise RuntimeError(f"Ollama API error: {response.status_code} {response.text}")

# Extract the dictionary from the LLM's response
import ast
llm_output = response.json()["response"]
try:
    # Try to parse the output as a Python dict
    granularity_hierarchy = ast.literal_eval(llm_output.strip())
except Exception as e:
    print("Failed to parse LLM output. Raw output:")
    print(llm_output)
    raise e

# Print the hierarchy
import pprint
print("\nGranularity Hierarchy for 'native-country' (from LLM):")
pprint.pprint(granularity_hierarchy)

Distinct values in column 'age':
['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '83', '84', '85', '89', '90', '?', '[10-19]', '[20-29]', '[30-39]', '[40-49]', '[50-59]', '[60-69]', '[70-79]', '[80-90]', 'alt', 'jung', 'mittel']
Number of unique values in 'age': 82
Distinct values in column 'workclass':
['?', 'Federal-gov', 'Local-gov', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay', 'gov', 'private_sector', 'self_emp', 'without_pay']
Number of unique values in 'workclass': 12
Distinct values in column 'fnlwgt':
['100009', '100032', '100054', '100099', '100125', '100135', '100219', '100235', '100292', '100294', '100301', '

SyntaxError: invalid syntax (<unknown>, line 1)